<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Fine_tuning_BERT__multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [ ]:
!pip install -q transformers datasets

     |████████████████████████████████| 4.4 MB 32.0 MB/s 
     |████████████████████████████████| 362 kB 56.3 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 6.6 MB 51.7 MB/s 
     |████████████████████████████████| 596 kB 69.9 MB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 212 kB 62.5 MB/s 
     |████████████████████████████████| 140 kB 47.2 MB/s 
     |████████████████████████████████| 127 kB 51.7 MB/s 
     |████████████████████████████████| 94 kB 705 kB/s 
     |████████████████████████████████| 271 kB 41.1 MB/s 
     |████████████████████████████████| 144 kB 53.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PaperA3/Multilabel/df_raw.csv')
df = df.drop('Unnamed: 0', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
df.head()

,HADM_ID,comment_text,L_10.0,L_8.0,L_6.0,L_4.0,L_11.0,L_5.0,L_2.0,L_18.0,L_1.0,L_14.0,L_3.0,L_9.0,L_17.0,L_15.0,L_13.0,L_16.0,L_7.0,L_12.0
0,157267.0,Mucous body substance Twice a day clarithromyc...,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,123103.0,Machine Done (qualifier value) Bromdimethoxyam...,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,139931.0,Structure of right renal artery Chemical injur...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,198295.0,Calculus of gallbladder with cholecystitis Hep...,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
4,177962.0,Aplastic Three Vessel Coronary Disease Scheuer...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
train_size=0.8
train, rem= train_test_split(df, train_size=0.8)
test_size = 0.5
valid, test = train_test_split(rem, test_size=0.5)

print(train.shape)
print(valid.shape)
print(test.shape)



(19630, 20)
(2454, 20)
(2454, 20)


In [ ]:
from datasets import Dataset
import datasets
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
valid_dataset = Dataset.from_pandas(valid)
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset,"valid":valid_dataset})

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_10.0', 'L_8.0', 'L_6.0', 'L_4.0', 'L_11.0', 'L_5.0', 'L_2.0', 'L_18.0', 'L_1.0', 'L_14.0', 'L_3.0', 'L_9.0', 'L_17.0', 'L_15.0', 'L_13.0', 'L_16.0', 'L_7.0', 'L_12.0', '__index_level_0__'],
        num_rows: 19630
    })
    test: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_10.0', 'L_8.0', 'L_6.0', 'L_4.0', 'L_11.0', 'L_5.0', 'L_2.0', 'L_18.0', 'L_1.0', 'L_14.0', 'L_3.0', 'L_9.0', 'L_17.0', 'L_15.0', 'L_13.0', 'L_16.0', 'L_7.0', 'L_12.0', '__index_level_0__'],
        num_rows: 2454
    })
    valid: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_10.0', 'L_8.0', 'L_6.0', 'L_4.0', 'L_11.0', 'L_5.0', 'L_2.0', 'L_18.0', 'L_1.0', 'L_14.0', 'L_3.0', 'L_9.0', 'L_17.0', 'L_15.0', 'L_13.0', 'L_16.0', 'L_7.0', 'L_12.0', '__index_level_0__'],
        num_rows: 2454
    })
})

Let's check the first example of the training split:

In [ ]:
example = dataset['train'][0]
example

{'HADM_ID': 162963.0,
 'L_1.0': False,
 'L_10.0': True,
 'L_11.0': False,
 'L_12.0': False,
 'L_13.0': False,
 'L_14.0': False,
 'L_15.0': False,
 'L_16.0': False,
 'L_17.0': False,
 'L_18.0': True,
 'L_2.0': False,
 'L_3.0': False,
 'L_4.0': False,
 'L_5.0': False,
 'L_6.0': False,
 'L_7.0': True,
 'L_8.0': False,
 'L_9.0': False,
 '__index_level_0__': 13124,
 'comment_text': 'Lymphoid leukemia Leptomeningeal enhancement R-CVP Regimen fludarabine Candida albicans R-CHOP Regimen Avastin Chronic Lymphocytic Leukemia Pulmonary venous congestion Conversions cytoxan Greece Neck stiffness Nasal congestion (finding) Osteopetrosis ampicillin fluconazole norepinephrine acyclovir Sinusitis Barrett Esophagus pentoxifylline Septic Shock azithromycin Nystagmus Splenomegaly Hypesthesia albuterol Deny (action) Weight decreased Weakness GEN1 gene Myalgia Dysuria Palpitations Heart murmur Lymphadenopathy Infarction Diarrhea Vomiting Nausea Sore to touch Patients Communicable Diseases Fever Chest Pain 

In [ ]:
example = dataset['train'][0]
example

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['HADM_ID', 'comment_text','__index_level_0__']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['L_10.0',
 'L_8.0',
 'L_6.0',
 'L_4.0',
 'L_11.0',
 'L_5.0',
 'L_2.0',
 'L_18.0',
 'L_1.0',
 'L_14.0',
 'L_3.0',
 'L_9.0',
 'L_17.0',
 'L_15.0',
 'L_13.0',
 'L_16.0',
 'L_7.0',
 'L_12.0']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["comment_text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f5056db1440> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] lymphoid leukemia leptomeningeal enhancement r - cvp regimen fludarabine candida albicans r - chop regimen avastin chronic lymphocytic leukemia pulmonary venous congestion conversions cytoxan greece neck stiffness nasal congestion ( finding ) osteopetrosis ampicillin fluconazole norepinephrine acyclovir sinusitis barrett esophagus pentoxifylline septic shock azithromycin nystagmus splenomegaly hypesthesia albuterol deny ( action ) weight decreased weakness [SEP]'

In [ ]:
example['labels']

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['L_10.0', 'L_18.0', 'L_7.0']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [ ]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 32
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1048, 24335,  8458,  9314, 25468,  3393, 13876,  8462,  5582,
        15879, 22415,  1054,  1011, 26226,  2361,  6939,  2078, 19857, 25329,
        16765, 27467,  2850,  2632, 13592,  6962,  1054,  1011, 24494,  6939,
         2078, 10927, 16643,  2078, 11888,  1048, 24335,  8458, 10085, 21252,
        25468, 21908,  2310, 18674, 20176, 25834, 22330,  3406, 18684,  2078,
         5483,  3300, 10551,  2791, 19077, 20176,  1006,  4531,  1007,  9808,
         2618, 17635, 13181,  6190, 23713, 28775, 21202, 19857, 24366,  6844,
         2571,  4496, 13699,  3170,  8458, 11467,  9353,  2100, 20464,  4492,
         4313,  8254,  2271, 13706, 12712,  9686,  7361,  3270, 12349,  7279,
         3406,  9048, 12031, 21202,  2063, 17419,  2594,  5213, 17207,  8939,
        21716,  2100, 15459,  6396,  9153, 21693,  2271, 11867,  7770,  8462,
         9692,  2100,  1044, 18863,  3367, 15689,  2401,  2632,  8569, 27833,
         9772,  1006,  2895,  1007,  3635, 10548, 11251,   102])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.6861, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[-0.0570,  0.0364,  0.0338, -0.1094, -0.0996, -0.0615, -0.4660, -0.1229,
                                    -0.0650, -0.2275, -0.3557, -0.0643,  0.0121,  0.0476,  0.4367,  0.3991,
                                     0.3018,  0.1266]], grad_fn=<AddmmBackward0>))])

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19630
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3070


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.298900,0.245531,0.364133,0.616358,0.139364
2,0.243600,0.231770,0.436618,0.650391,0.161369
3,0.228800,0.226121,0.465714,0.665215,0.171149
4,0.220800,0.223889,0.484130,0.675561,0.184189
5,0.207300,0.223787,0.480508,0.673869,0.179707


***** Running Evaluation *****
  Num examples = 2454
  Batch size = 32
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-614
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-614/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-614/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-614/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-614/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2454
  Batch size = 32
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-1228
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-1228/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-1228/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-1228/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/check

TrainOutput(global_step=3070, training_loss=0.2346497936435165, metrics={'train_runtime': 2225.462, 'train_samples_per_second': 44.103, 'train_steps_per_second': 1.379, 'total_flos': 6457014986572800.0, 'train_loss': 0.2346497936435165, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2454
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.18418907905460472,
 'eval_f1': 0.4841303427845958,
 'eval_loss': 0.22388894855976105,
 'eval_roc_auc': 0.6755605868840684,
 'eval_runtime': 18.1168,
 'eval_samples_per_second': 135.454,
 'eval_steps_per_second': 4.25}

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "Lymphoid leukemia Leptomeningeal enhancement R-CVP Regimen fludarabine Candida albicans R-CHOP Regimen Avastin Chronic Lymphocytic Leukemia Pulmonary venous congestion Conversions cytoxan Greece Neck stiffness Nasal congestion (finding) Osteopetrosis ampicillin fluconazole norepinephrine acyclovir Sinusitis Barrett Esophagus pentoxifylline Septic Shock azithromycin Nystagmus Splenomegaly Hypesthesia albuterol Deny (action) Weight decreased Weakness GEN1 gene Myalgia"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 18])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['L_10.0', 'L_18.0']
